<a href="https://colab.research.google.com/github/mongvuot336699/DevC/blob/master/Copy_of_Weekly_Project_BBC_Articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Organize ML projects with Scikit-Learn

While Machine Learning is powerful, people often overestimate it: apply machine learning to your project, and all your problems will be solved. In reality, it's not this simple. To be effective, one needs to organize the work very well. In this notebook, we will walkthrough practical aspects of a ML project. To look at the big picture, let's start with a checklist below. It should work reasonably well for most ML projects, but make sure to adapt it to your needs:

1. **Define the scope of work and objective**
    * How is your solution be used?
    * How should performance be measured? Are there any contraints?
    * How would the problem be solved manually?
    * List the available assumptions, and verify if possible.
    
    
2. **Get the data**
    * Document where you can get that data
    * Store data in a workspace you can easily access
    * Convert the data to a format you can easily manipulate
    * Check the overview (size, type, sample, description, statistics)
    * Data cleaning
    
    
3. **EDA & Data transformation**
    * Study each attribute and its characteristics (missing values, type of distribution, usefulness)
    * Visualize the data
    * Study the correlations between attributes
    * Feature selection, Feature Engineering, Feature scaling
    * Write functions for all data transformations
    
    
4. **Train models**
    * Automate as much as possible
    * Train promising models quickly using standard parameters. Measure and compare their performance
    * Analyze the errors the models make
    * Shortlist the top three of five most promising models, preferring models that make different types of errors.


5. **Fine-tunning**
    * Treat data transformation choices as hyperparameters, expecially when you are not sure about them (e.g., replace missing values with zeros or with the median value)
    * Unless there are very few hyperparameter value to explore, prefer random search over grid search.
    * Try ensemble methods
    * Test your final model on the test set to estimate the generalizaiton error. Don't tweak your model again, you would start overfitting the test set.

## Example: Articles categorization

### Objectives

Build a model to determine the categories of articles. 

### Get Data

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")

In [3]:
bbc = pd.read_csv('https://raw.githubusercontent.com/dhminh1024/practice_datasets/master/bbc-text.csv')

In [4]:
bbc.sample(5)

,category,text
2144,entertainment,rock group korn s guitarist quits the guitaris...
1504,entertainment,global release for japan hit film oscar-winnin...
669,entertainment,double win for sea inside spanish movie the se...
354,entertainment,neeson in bid to revive theatre hollywood film...
1439,sport,moore questions captaincy brian moore believes...


In [ ]:
bbc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [ ]:
bbc['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [ ]:
bbc['text'][0]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high

In [ ]:
# Clean data
# xoa - + . , 
# stopword - loc bo cac chu thua 
# stemming

In [5]:
import re

def prepro(text):
  text = re.sub('[^a-z]',' ',text)
  return text

bbc['text'] = bbc['text'].apply(prepro)

In [ ]:
bbc['text'][0]

'tv future in the hands of viewers with home theatre systems  plasma high definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time   that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes  with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices   one of the most talked about technologies of ces has been digital and personal video recorders  dvr and pvr   these set top boxes  like the us s tivo and the uk s sky  system  allow people to record  store  play  pause and forward wind tv programmes when they want   essentially  the technology allows for much more personalised tv  they are also being built in to high

In [6]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

In [7]:
from collections import Counter

vocab = Counter()
for row in bbc['text']:
  for word in row.split(' '):
    vocab[word] += 1

vocab.most_common(5)

[('', 188374), ('the', 52636), ('to', 25113), ('of', 20008), ('and', 18612)]

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from nltk.corpus import stopwords
stopWords = stopwords.words('english')

vocab_reduce = Counter()
for w, c in vocab.items():
  if w not in stopWords :
    vocab_reduce[w] = c

vocab_reduce.most_common(5)

[('', 188374), ('said', 7255), ('mr', 3005), ('would', 2581), ('year', 2310)]

In [10]:
from sklearn.model_selection import train_test_split

X = bbc['text']
y = bbc['category']

X_train , X_test , y_train , y_test = train_test_split(X,y, test_size = 0.2, random_state =42)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf =  TfidfVectorizer(
    stop_words = stopWords,
    tokenizer = tokenizer_porter
)

In [13]:
tfidf.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenizer_porter at 0x7fb47a339730>,
                use_idf=True, vocabulary=None)

In [15]:
X_train_tf = tfidf.transform(X_train)

In [14]:
X_test_tf = tfidf.transform(X_test)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.metrics import accuracy_score

dict_model = {
    'lr': LogisticRegression(),
    'knn': KNeighborsClassifier(),
    'dct': DecisionTreeClassifier(),
    'rdtc': RandomForestClassifier(),
    'GBCF': GradientBoostingClassifier()
}

for key in dict_model:
  model = dict_model[key].fit(X_train_tf,y_train)
  y_train_pre = model.predict(X_train_tf)
  y_test_pre = model.predict(X_test_tf)
  print(f'Accuracy_score of {key} train : \n' , accuracy_score(y_train, y_train_pre))
  print(f'Accuracy_score of {key} test : \n' , accuracy_score(y_test, y_test_pre))

Accuracy_score of lr train : 
 0.996629213483146
Accuracy_score of lr test : 
 0.9640449438202248
Accuracy_score of knn train : 
 0.9595505617977528
Accuracy_score of knn test : 
 0.9370786516853933
Accuracy_score of dct train : 
 1.0
Accuracy_score of dct test : 
 0.7910112359550562
Accuracy_score of rdtc train : 
 1.0
Accuracy_score of rdtc test : 
 0.950561797752809
Accuracy_score of GBCF train : 
 1.0
Accuracy_score of GBCF test : 
 0.946067415730337


In [25]:
from sklearn.model_selection import GridSearchCV 

lr2 = LogisticRegression()

In [27]:
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(lr2,param_grid,cv = 4 )
grid.fit(X_train_tf, y_train)
print("Best estimator:\n{}".format(grid.best_estimator_))

ValueError: ignored